## 1차 데이터셋 다운로드

In [11]:
!pip install yfinance

import yfinance as yf
import pandas as pd

In [4]:
# 1. 티커 리스트 설정
ticker_list = ['^KS11', '^GSPC', 'KRW=X', '^TNX', '^VIX']

In [5]:
# 2. 데이터 다운로드(20년치, 일봉)
KOSPI_20y_daily = yf.download(ticker_list, 
                   start='2005-01-01', 
                   interval='1d')

print("다운로드 원본 데이터 : ")
print(KOSPI_20y_daily.head()) 

/var/folders/51/80r302kd7fn1ly1211v74g000000gn/T/ipykernel_25997/779961824.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  KOSPI_20y_daily = yf.download(ticker_list,
[*********************100%***********************]  5 of 5 completed

다운로드 원본 데이터 : 
Price             Close                                                High  \
Ticker            KRW=X        ^GSPC       ^KS11   ^TNX   ^VIX        KRW=X   
Date                                                                          
2005-01-03  1027.300049  1202.079956  893.710022  4.220  14.08  1058.400024   
2005-01-04  1028.699951  1188.050049  886.900024  4.283  13.98  1036.949951   
2005-01-05  1035.800049  1183.739990  885.190002  4.277  14.09  1045.000000   
2005-01-06  1048.400024  1187.890015  871.280029  4.272  13.58  1056.900024   
2005-01-07  1045.099976  1186.189941  870.840027  4.285  13.49  1049.750000   

Price                                              ...         Open  \
Ticker            ^GSPC       ^KS11   ^TNX   ^VIX  ...        KRW=X   
Date                                               ...                
2005-01-03  1217.800049  897.590027  4.270  14.23  ...  1044.199951   
2005-01-04  1205.839966  895.400024  4.307  14.45  ...  1027.099976 

In [7]:
# 3. 데이터 정리(가격 데이터와 거래량 데이터 분리)
df_price = pd.DataFrame()
df_price['KOSPI_Close'] = KOSPI_20y_daily[('Close', '^KS11')]
df_price['KOSPI_Open'] = KOSPI_20y_daily[('Open', '^KS11')]
df_price['KOSPI_High'] = KOSPI_20y_daily[('High', '^KS11')]
df_price['KOSPI_Low'] = KOSPI_20y_daily[('Low', '^KS11')]

## 'Adj Close' -> 'Close'로 수정.
df_price['SP500'] = KOSPI_20y_daily[('Close', '^GSPC')]
df_price['USD_KRW'] = KOSPI_20y_daily[('Close', 'KRW=X')]
df_price['US_10Y_Bond'] = KOSPI_20y_daily[('Close', '^TNX')]
df_price['VIX'] = KOSPI_20y_daily[('Close', '^VIX')]

## 거래량 데이터
df_volume = pd.DataFrame()
df_volume['KOSPI_Volume'] = KOSPI_20y_daily[('Volume', '^KS11')]


In [8]:
# 4. 결측치 처리
# 가격 데이터는 직전 값으로 채우기
df_price_filled = df_price.fillna(method='ffill')

# 거래량 데이터는 0으로 채우기
df_volume_filled = df_volume.fillna(0)

/var/folders/51/80r302kd7fn1ly1211v74g000000gn/T/ipykernel_25997/1528464037.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_price_filled = df_price.fillna(method='ffill')


In [9]:
# 5. 최종 데이터셋으로 병합
df_final = pd.concat([df_price_filled, df_volume_filled], axis=1)

# 6. 맨 처음 값들(bfill로 채워야 함)도 마저 채우기
df_final = df_final.fillna(method='bfill')

print("최종 전처리 완료된 데이터 : ")
df_final.info() # NaN이 없는지 확인
print(df_final.head())

최종 전처리 완료된 데이터 : 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5437 entries, 2005-01-03 to 2025-11-05
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   KOSPI_Close   5437 non-null   float64
 1   KOSPI_Open    5437 non-null   float64
 2   KOSPI_High    5437 non-null   float64
 3   KOSPI_Low     5437 non-null   float64
 4   SP500         5437 non-null   float64
 5   USD_KRW       5437 non-null   float64
 6   US_10Y_Bond   5437 non-null   float64
 7   VIX           5437 non-null   float64
 8   KOSPI_Volume  5437 non-null   float64
dtypes: float64(9)
memory usage: 424.8 KB
            KOSPI_Close  KOSPI_Open  KOSPI_High   KOSPI_Low        SP500  \
Date                                                                       
2005-01-03   893.710022  896.000000  897.590027  890.929993  1202.079956   
2005-01-04   886.900024  890.559998  895.400024  884.940002  1188.050049   
2005-01-05   885.190002  874.919983  88

/var/folders/51/80r302kd7fn1ly1211v74g000000gn/T/ipykernel_25997/126901464.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final = df_final.fillna(method='bfill')


In [ ]:
# 7. 최종 CSV 파일로 저장
df_final.to_csv("kospi_20y_daily.csv")

print("파일 저장 완료")

파일 저장 완료
